# Average Consensus

## Background the problem setting

Find $\bar{x} = \sum(x_i)$ for all agent $i$ defined over graph.
Application example? Page-rank? Markov-chain?


## Graph Definition

Talk about 
- graph definition $\mathcal{G} = (V,E)$
- directional and undirectional graph
- incoming neighbors and outgoing neighbors
- Stochastic matrix

BlueFog about

- `bf.set_topology()`
- Common topologies examples
- Weights and is_weighted Setting

## Communication over graph and Consensus algorithm

Algorithm:
- simply $x_{k+1} = W x_k$ iteration

BlueFog 
- `bf.neighbor_allreduce()`
- Collective communication -- BSP pattern (Bulk synchronized processing)

Analysis:
- Spectrum of Combination matrix $W$
- Convergence proof and speed discussion